PROJECT - Yhills

 ##Movie Recommendation System##

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv('/content/movies.csv')

In [ ]:
df1 = pd.read_csv("/content/credits.csv")

In [ ]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [ ]:
df1.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [ ]:
# merging two different dataframes based on any same column
df2 = df.merge(df1, on='title')

In [ ]:
df2.shape

(4808, 23)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4808 entries, 0 to 4807
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4808 non-null   int64  
 1   genres                4808 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4808 non-null   int64  
 4   keywords              4808 non-null   object 
 5   original_language     4808 non-null   object 
 6   original_title        4808 non-null   object 
 7   overview              4805 non-null   object 
 8   popularity            4808 non-null   float64
 9   production_companies  4808 non-null   object 
 10  production_countries  4808 non-null   object 
 11  release_date          4807 non-null   object 
 12  revenue               4808 non-null   int64  
 13  runtime               4806 non-null   float64
 14  spoken_languages      4808 non-null   object 
 15  status               

In [ ]:
features = ['id','genres','tagline','keywords','overview','title','cast','crew']

In [ ]:
df2 = df2[['id','genres','tagline','keywords','overview','title','cast','crew']]

In [ ]:
df2.head(2)

,id,genres,tagline,keywords,overview,title,cast,crew
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Enter the World of Pandora.,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","At the end of the world, the adventure begins.","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [ ]:
df2.isnull().sum()

id            0
genres        0
tagline     844
keywords      0
overview      3
title         0
cast          0
crew          0
dtype: int64

In [ ]:
# replacing the missing values with null strings

for i in features:
  df2[i] = df2[i].fillna('')

In [ ]:
df2.isnull().sum()

id          0
genres      0
tagline     0
keywords    0
overview    0
title       0
cast        0
crew        0
dtype: int64

In [ ]:
df2.dropna(inplace=True)

In [ ]:
df2.isnull().sum()

id          0
genres      0
tagline     0
keywords    0
overview    0
title       0
cast        0
crew        0
dtype: int64

In [ ]:
df2.duplicated().sum()

0

In [ ]:
df2.iloc[0]

id                                                      19995
genres      [{"id": 28, "name": "Action"}, {"id": 12, "nam...
tagline                           Enter the World of Pandora.
keywords    [{"id": 1463, "name": "culture clash"}, {"id":...
overview    In the 22nd century, a paraplegic Marine is di...
title                                                  Avatar
cast        [{"cast_id": 242, "character": "Jake Sully", "...
crew        [{"credit_id": "52fe48009251416c750aca23", "de...
Name: 0, dtype: object

In [ ]:
def convert(obj):
  list = []
  for i in ast.literal_eval(obj):
    list.append(i['name'])
  return list

In [ ]:
import ast

In [ ]:
df2['genres'] = df2['genres'].apply(convert)

In [ ]:
df2['keywords'] = df2['keywords'].apply(convert)

In [ ]:
df2.head(2)

,id,genres,tagline,keywords,overview,title,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]",Enter the World of Pandora.,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[Adventure, Fantasy, Action]","At the end of the world, the adventure begins.","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [ ]:
df2['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [ ]:
def convert2(obj):
  list = []
  c = 0
  for i in ast.literal_eval(obj):
    if c!=4:
      list.append(i['name'])
      c = c+1
    else:
      break
  return list

In [ ]:
df2['cast'] = df2['cast'].apply(convert2)

In [ ]:
def director(obj):
  list = []
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      list.append(i['name'])
      break
  return list


In [ ]:
df2['crew'] = df2['crew'].apply(director)

In [ ]:
df2.head(2)

,id,genres,tagline,keywords,overview,title,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]",Enter the World of Pandora.,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,"[Adventure, Fantasy, Action]","At the end of the world, the adventure begins.","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4808 entries, 0 to 4807
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        4808 non-null   int64 
 1   genres    4808 non-null   object
 2   tagline   4808 non-null   object
 3   keywords  4808 non-null   object
 4   overview  4808 non-null   object
 5   title     4808 non-null   object
 6   cast      4808 non-null   object
 7   crew      4808 non-null   object
dtypes: int64(1), object(7)
memory usage: 467.1+ KB


In [ ]:
df2['overview'] = df2['overview'].apply(lambda x:x.split())

In [ ]:
df2['tagline'] = df2['tagline'].apply(lambda x:x.split())

In [ ]:
df2.head(2)

,id,genres,tagline,keywords,overview,title,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[Enter, the, World, of, Pandora.]","[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,"[Adventure, Fantasy, Action]","[At, the, end, of, the, world,, the, adventure...","[ocean, drug abuse, exotic island, east india ...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]


In [ ]:
df2['genres'] = df2['genres'].apply(lambda x: [i.replace(" ","") for i in x])
df2['tagline'] = df2['tagline'].apply(lambda x: [i.replace(" ","") for i in x])
df2['keywords'] = df2['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
df2['cast'] = df2['cast'].apply(lambda x: [i.replace(" ","") for i in x])
df2['crew'] = df2['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
df2.head(5)

,id,genres,tagline,keywords,overview,title,cast,crew
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]","[Enter, the, World, of, Pandora.]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,285,"[Adventure, Fantasy, Action]","[At, the, end, of, the, world,, the, adventure...","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,206647,"[Action, Adventure, Crime]","[A, Plan, No, One, Escapes]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,49026,"[Action, Crime, Drama, Thriller]","[The, Legend, Ends]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,49529,"[Action, Adventure, ScienceFiction]","[Lost, in, our, world,, found, in, another.]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]


In [ ]:
df2['desc'] = df2['genres'] + df2['tagline'] + df2['keywords'] + df2['overview'] + df2['cast'] + df2['crew']

In [ ]:
df2.head()

,id,genres,tagline,keywords,overview,title,cast,crew,desc
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]","[Enter, the, World, of, Pandora.]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction, E..."
1,285,"[Adventure, Fantasy, Action]","[At, the, end, of, the, world,, the, adventure...","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski],"[Adventure, Fantasy, Action, At, the, end, of,..."
2,206647,"[Action, Adventure, Crime]","[A, Plan, No, One, Escapes]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes],"[Action, Adventure, Crime, A, Plan, No, One, E..."
3,49026,"[Action, Crime, Drama, Thriller]","[The, Legend, Ends]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan],"[Action, Crime, Drama, Thriller, The, Legend, ..."
4,49529,"[Action, Adventure, ScienceFiction]","[Lost, in, our, world,, found, in, another.]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton],"[Action, Adventure, ScienceFiction, Lost, in, ..."


In [ ]:
new_df =  df2[['id','title','desc']]

In [ ]:
new_df.head()

,id,title,desc
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, E..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, At, the, end, of,..."
2,206647,Spectre,"[Action, Adventure, Crime, A, Plan, No, One, E..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, The, Legend, ..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, Lost, in, ..."


In [ ]:
new_df['desc'] = new_df['desc'].apply(lambda x:" ".join(x))

<ipython-input-282-91d825b108a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['desc'] = new_df['desc'].apply(lambda x:" ".join(x))


In [ ]:
new_df.head()

,id,title,desc
0,19995,Avatar,Action Adventure Fantasy ScienceFiction Enter ...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action At the end of the wor...
2,206647,Spectre,Action Adventure Crime A Plan No One Escapes s...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller The Legend Ends dc...
4,49529,John Carter,Action Adventure ScienceFiction Lost in our wo...


In [ ]:
new_df['desc'][0]


'Action Adventure Fantasy ScienceFiction Enter the World of Pandora. cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. SamWorthington ZoeSaldana SigourneyWeaver StephenLang JamesCameron'

In [ ]:
new_df['desc'] = new_df['desc'].apply(lambda x:x.lower())

<ipython-input-285-e1cf692b56ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['desc'] = new_df['desc'].apply(lambda x:x.lower())


In [ ]:
 import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
  y = []

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [ ]:
new_df.head()

,id,title,desc
0,19995,Avatar,action adventure fantasy sciencefiction enter ...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action at the end of the wor...
2,206647,Spectre,action adventure crime a plan no one escapes s...
3,49026,The Dark Knight Rises,action crime drama thriller the legend ends dc...
4,49529,John Carter,action adventure sciencefiction lost in our wo...


In [ ]:
new_df['desc'] = new_df['desc'].apply(stem)

<ipython-input-290-8c69cd86a14b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['desc'] = new_df['desc'].apply(stem)


In [ ]:
new_df.shape

(4808, 3)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors= cv.fit_transform(new_df['desc']).toarray()

In [ ]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity

array([[1.        , 0.11908138, 0.07733089, ..., 0.06406221, 0.        ,
        0.        ],
       [0.11908138, 1.        , 0.07919455, ..., 0.0437374 , 0.02033209,
        0.02354408],
       [0.07733089, 0.07919455, 1.        , ..., 0.02366905, 0.        ,
        0.        ],
       ...,
       [0.06406221, 0.0437374 , 0.02366905, ..., 1.        , 0.03646019,
        0.04222003],
       [0.        , 0.02033209, 0.        , ..., 0.03646019, 1.        ,
        0.09813358],
       [0.        , 0.02354408, 0.        , ..., 0.04222003, 0.09813358,
        1.        ]])

In [ ]:
list_titles = (new_df['title']).tolist()

In [ ]:
print(list_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

[(1216, 0.25900594339123895),
 (507, 0.24781017679048262),
 (582, 0.24675483634579948),
 (3731, 0.2448989999894618),
 (2409, 0.23964061052266444)]

In [ ]:
def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]
  distances = similarity[movie_index]
  movie_list = sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

  for i in movie_list:
    print(new_df.iloc[i[0]].title)

In [ ]:
movie = input('Enter the movie name:')

Enter the movie name:avatar


In [ ]:
#finding best match to the one given by user
close_match = difflib.get_close_matches(movie, list_titles)
print(close_match)

['Avatar']


In [ ]:
best =close_match[0]
print(best)
recommend(best)

Avatar
Aliens vs Predator: Requiem
Independence Day
Battle: Los Angeles
Falcon Rising
Aliens


In [ ]:
import difflib

# Example data
movie = "Star Wars"
list_titles = ["Star Wars: Episode IV - A New Hope", "Star Trek", "Star Wars: The Empire Strikes Back"]

# Finding the best match to 'movie' from 'list_titles'
close_match = difflib.get_close_matches(movie, list_titles)
print(close_match)

['Star Trek']
